<div id="TOP" />
<a href="#END" > END of Notebook (filter) </a>

In [1]:
cd ~/nb/2022-Terraform/content/tf-labs/4b.TerraformMaps/

# -- CONFIG ------------------------------------
LAB_TAG=${PWD##*/} 
LAB_NAME="TFI_Azure_Lab4b.TerraformMaps"
LAB_WEIGHT=42
LAB_MODE="Terraform"
export MAX_LINE_LEN=80
# ----------------------------------------------

DIE() { echo "[$PWD] Error: $*"; banner "ERROR"; exit 1; }
NBTOOL_RC=~/scripts/nbtool.rc; [ ! -f $NBTOOL_RC ] && DIE "No such script as $NBTOOL_RC"
echo "==== Reading nbtool functions from '$NBTOOL_RC' ===="; . $NBTOOL_RC
EXCL_FN_INIT_NOTEBOOK
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

bash: cd: /home/student/nb/2022-Terraform/content/tf-labs/4b.TerraformMaps/: No such file or directory
==== Reading nbtool functions from '/home/student/scripts/nbtool.rc' ====
TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.

******** Mon 09 Jan 2023 02:43:40 PM 

#EXCLUDE
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

NB_DIR=$PWD
rm -rf ~/tmp/terraform*
#rm -rf ~/labs

FRONTMATTER=$(cat <<END_HEREDOC
---
title: 'Lab 4b.TerraformMaps'
date: 2019-02-11T19:27:37+10:00
weight: 42
---

END_HEREDOC
)

echo; echo "==== Reading nbtool functions ===="
. ~/bin/nbtool.rc

echo; echo "==== Setting Exercise tag ===="
LABTAG=${PWD##*/}  #LAB="4b.TerraformMaps"
echo "LABTAG=$LAB_TAG" | EXCL_FN_HIGHLIGHT_EOL =

echo; echo "==== Setting TF_DATA_DIR ===="
export TF_DATA_DIR=~/dot.terraform; echo "TF_DATA_DIR=$TF_DATA_DIR" | EXCL_FN_HIGHLIGHT_EOL =

NO_EXEC echo SHOULD NOT BE SEEN | EXCL_FN_HIGHLIGHT_SOL SHOULD
EXEC    echo SHOULD BE SEEN     | EXCL_FN_HIGHLIGHT_SOL SHOULD

# echo; # echo "==== Filtering current notebook ===="; EXCL_FN_FILTER_NOTEBOOK index.ipynb

echo; echo "==== Setting AWS vars for student20 account ===="
. ~/.aws/linked/student20.rc; env | grep MYAWS_ACCOUNT_NAME= | EXCL_FN_HIGHLIGHT_EOL =

# --INCLUDE--SECTION--

In [2]:
EXCL_FN_LAB_ENV 0

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

## Background:

Here, we learn how to create and use terraform maps. Maps are key/value pairs which we can create and look up as we need to in our templates.

## Tasks:
### 1. Make a directory called ‘lab4b’ underneath the labs directory.
### 2. Change into the directory.
### 3. Create the following files: main.tf, resources.tf vars.tf.

The main.tf file should contain:

In [3]:
#EXCLUDE

mkdir -p ~/labs/lab4b
cd ~/labs/lab4b


cat > provider.tf <<EOF

terraform {
  required_version = ">= 1.1.0"

  required_providers {
    aws = {
      version = "~> 4.0"
    }
  }
}

provider "aws" {
  region = var.region

  default_tags {
    tags = {
      Environment = "Terraform Introduction"
    }
  }
}

EOF


cat > resources.tf <<EOF

resource "aws_vpc" "main_vpc" {
  cidr_block       = var.vpc_cidr
  instance_tenancy = "default"

  tags = {
    Name           = "Main"
    Location       = "London"
    LabName        = "$LAB_TAG"
  }
}

resource "aws_subnet" "vpc_subnets" {
  count = length(var.vpc_subnet_cidr)

  vpc_id             = aws_vpc.main_vpc.id

  #cidr_block        = element(var.vpc_subnet_cidr,count.index)
  cidr_block         = var.vpc_subnet_cidr[count.index]

  #availability_zone = element(var.aaz[var.region],count.index)
  availability_zone  = var.aaz[var.region][count.index]

  tags = {
    Name = "subnet-\${count.index+1}"
    LabName = "$LAB_TAG"
  }
}
EOF

cat > vars.tf <<EOF

variable "region" {
  default = "us-west-1"
}
variable "vpc_cidr" {
  default = "192.168.0.0/16"
}
variable "vpc_subnet_cidr" {
  type = list
  # Reduced to 2-elements each due to non-availability of us-west-1a:
  #default = ["192.168.100.0/24","192.168.101.0/24","192.168.102.0/24"]
  default = ["192.168.101.0/24","192.168.102.0/24"]
}
variable "ami_instance" {
  type = map
  default = {
    "us-east-1" = "ami-0ac019f4fcb7cb7e6"
    "us-east-2" = "ami-0f65671a86f061fcd"
    "us-west-1" = "ami-063aa838bd7631e0b"
  }
}
variable "ami_instance_type" {
  default = "t2.micro"
}
variable "aaz" {
  type = map
  default = {
    #"us-east-1" = ["us-east-1a","us-east-1b","us-east-1c"]
    #"us-east-2" = ["us-east-2a","us-east-2b","us-east-2c"]
    #"us-west-1" = ["us-west-1a","us-west-1b","us-west-1c"]

    # Reduced to 2-elements each due to non-availability of us-west-1a:
    #"us-east-1" = ["us-east-1b","us-east-1c"]
    #"us-east-2" = ["us-east-2b","us-east-2c"]
    "us-west-1" = ["us-west-1b","us-west-1c"]
  }
} 
EOF

cat > outputs.tf <<EOF
output subnets { value = aws_subnet.vpc_subnets[*].cidr_block }
output zones   { value = aws_subnet.vpc_subnets[*].availability_zone }
EOF

The provider.tf file should contain:

In [4]:
cat provider.tf


terraform {
  required_version = ">= 1.1.0"

  required_providers {
    aws = {
      version = "~> 4.0"
    }
  }
}

provider "aws" {
  region = var.region

  default_tags {
    tags = {
      Environment = "Terraform Introduction"
    }
  }
}



The resources.tf file should contain:

In [5]:
cat resources.tf


resource "aws_vpc" "main_vpc" {
  cidr_block       = var.vpc_cidr
  instance_tenancy = "default"

  tags = {
    Name           = "Main"
    Location       = "London"
    LabName        = "4b.TerraformMaps"
  }
}

resource "aws_subnet" "vpc_subnets" {
  count = length(var.vpc_subnet_cidr)

  vpc_id             = aws_vpc.main_vpc.id

  #cidr_block        = element(var.vpc_subnet_cidr,count.index)
  cidr_block         = var.vpc_subnet_cidr[count.index]

  #availability_zone = element(var.aaz[var.region],count.index)
  availability_zone  = var.aaz[var.region][count.index]

  tags = {
    Name = "subnet-${count.index+1}"
    LabName = "4b.TerraformMaps"
  }
}


and your vars.tf file should contain:

In [6]:
cat vars.tf


variable "region" {
  default = "us-west-1"
}
variable "vpc_cidr" {
  default = "192.168.0.0/16"
}
variable "vpc_subnet_cidr" {
  type = list
  # Reduced to 2-elements each due to non-availability of us-west-1a:
  #default = ["192.168.100.0/24","192.168.101.0/24","192.168.102.0/24"]
  default = ["192.168.101.0/24","192.168.102.0/24"]
}
variable "ami_instance" {
  type = map
  default = {
    "us-east-1" = "ami-0ac019f4fcb7cb7e6"
    "us-east-2" = "ami-0f65671a86f061fcd"
    "us-west-1" = "ami-063aa838bd7631e0b"
  }
}
variable "ami_instance_type" {
  default = "t2.micro"
}
variable "aaz" {
  type = map
  default = {
    #"us-east-1" = ["us-east-1a","us-east-1b","us-east-1c"]
    #"us-east-2" = ["us-east-2a","us-east-2b","us-east-2c"]
    #"us-west-1" = ["us-west-1a","us-west-1b","us-west-1c"]

    # Reduced to 2-elements each due to non-availability of us-west-1a:
    #"us-east-1" = ["us-east-1b","us-east-1c"]
    #"us-east-2" = ["us-east-2b","us-east-2c"]
    "us-west-1" = ["us-west-1

You may copy the outputs.tf file you used for the previous lab to see the output.

<!-- data "aws_availability_zones" "aaz" {} -->

### 4. Study the configuration files

Study the configuration files, taking the time to understand

* the use of the "vpc_subnet_cidr" list variable

* the use of the "ami_instance" map variable

* the use of the "aaz" map variable

- why we need to modify the list/map dimensions when an availability zone is not available

### 5. The configuration when visualized should look like

<div>
    <object data="graph.svg" type="image/svg+xml">
    </object>
</div>

![](graph.svg)

### 6. Initialize the configuration

This time we will set the TF_LOG variable to TRACE to see debugging output: ```TF_LOG=TRACE terraform init```

In [7]:
TF_LOG=TRACE terraform init

2023-01-09T14:44:37.219Z [INFO]  Terraform version: 1.3.7
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/go-tfe v1.9.0
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/hcl/v2 v2.15.0
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/terraform-config-inspect v0.0.0-20210209133302-4fd17a0faac2
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/terraform-svchost v0.0.0-20200729002733-f050f53b9734
2023-01-09T14:44:37.219Z [DEBUG] using github.com/zclconf/go-cty v1.12.1
2023-01-09T14:44:37.219Z [INFO]  Go runtime version: go1.19.4
2023-01-09T14:44:37.219Z [INFO]  CLI args: []string{"terraform", "init"}
2023-01-09T14:44:37.219Z [TRACE] Stdout is a terminal of width 80
2023-01-09T14:44:37.219Z [TRACE] Stderr is a terminal of width 80
2023-01-09T14:44:37.219Z [TRACE] Stdin is a terminal
2023-01-09T14:44:37.219Z [DEBUG] Attempting to open CLI config file: /home/student/.terraformrc
2023-01-09T14:44:37.219Z [DEBUG] File doesn't exist, but doesn't need t

selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


This command initializes the terraform directory structure.

Take the time to look at the trace output to see what the *init* process does and where it looks for and stores provider plugin or module files.

This can be particularly useful when build or installing third-party provider plugins manually - to debug possible failure to find the appropriate binaries.

### 7.  Preview the configuration

In [8]:
terraform plan


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_subnet.vpc_subnets[0] will be created
  + resource "aws_subnet" "vpc_subnets" {
      + arn                                            = (known after apply)
      + assign_ipv6_address_on_creation                = false
      + availability_zone                              = "us-west-1b"
      + availability_zone_id                           = (known after apply)
      + cidr_block                                     = "192.168.101.0/24"
      + enable_dns64                                   = false
      + enable_resource_name_dns_a_record_on_launch    = false
      + enable_resource_name_dns_aaaa_record_on_launch = false
      + id                                             = (known after apply)
      + ipv6_cidr_block_association_id                 = (known after apply)
      +

**verify** the actions that terraform will take.

### 8. Apply the configuration

In [9]:
TF_APPLY


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_subnet.vpc_subnets[0] will be created
  + resource "aws_subnet" "vpc_subnets" {
      + arn                                            = (known after apply)
      + assign_ipv6_address_on_creation                = false
      + availability_zone                              = "us-west-1b"
      + availability_zone_id                           = (known after apply)
      + cidr_block                                     = "192.168.101.0/24"
      + enable_dns64                                   = false
      + enable_resource_name_dns_a_record_on_launch    = false
      + enable_resource_name_dns_aaaa_record_on_launch = false
      + id                                             = (known after apply)
      + ipv6_cidr_block_association_id                 = (known after apply)
      +

Assuming that this works correctly, the AWS Provider will create a VPC, and 2 subnets located in 2 different
availability zones, and an ami instance running on each subnet.

### 9. Create some instances using the vpcs

#### Known error to be fixed ...

Getting error: ```Error launching source instance: InvalidParameter: Security group sg-07cbbd1042f3c81b3 and subnet subnet-2c6ba676 belong to different networks.```


Now add some new "aws_instance" resources, i.e. VMs, using count to create several.

Output the assigned public_ips of the VMs

Output also a combined output of public_ip, public_dns, private_ip for each VM of the form:
- "public address: <public_ip>[<public_dns>] private address: <private_ip>"

You should use the "*for*" expression to achieve this

In [10]:
#EXCLUDE

grep -q key_name vars.tf || cat >> vars.tf <<EOF
variable key_name {
  description = "AWS key_pair name to create"
  default     = "UNSET"
}
EOF

#echo; echo "=========== OK1 ========================="

cat > terraform.tfvars <<EOF
key_name = "LAB4b-key"
EOF

#echo; echo "=========== OK2 ========================="

cat > aws_instance.tf <<EOF

# An aws_instance is an 'AWS EC2' resource: a virtual machine
# In our config we can refer to this instance as:
#     aws_instance.example
#
resource "aws_instance" "example" {
  count         = 2

  ami           = "ami-00a3e4424e9ab3e56" # us-west-1
  instance_type = "t2.micro"

  tags     = { LabName = "$LAB_NAME" }
  vpc_security_group_ids = [aws_security_group.secgroup-ssh.id]
  key_name = aws_key_pair.generated_key.key_name
}

resource "aws_security_group" "secgroup-ssh" {
  name   = "simple security group - for ssh Ingress only"
  vpc_id = aws_vpc.main_vpc.id
  
  # Enable incoming ssh connection:
  ingress {
    from_port   = "22"
    to_port     = "22"
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }
}

# Create a TLS private key  resource
#
resource "tls_private_key" "mykey" {
  algorithm = "RSA"
  rsa_bits  = 4096
}

# Create an AWS 'key pair' resource usable for ssh from the TLS private key
#
resource "aws_key_pair" "generated_key" {
  key_name   = var.key_name
  public_key = tls_private_key.mykey.public_key_openssh
}

EOF

#echo; echo "=========== OK3 ========================="

cat > outputs.tf <<EOF
output "ssh_rsa_pub_key" { value = tls_private_key.mykey.public_key_openssh }

# NOTE: we mark this element as sensitive information,
#       to be able to output the value
output "ssh_pem_key" {
  value     = tls_private_key.mykey.private_key_pem
  sensitive = true
}

output "public_ip" { value = aws_instance.example.*.public_ip }

EOF

#echo; echo "=========== OK4 ========================="

In [11]:
#EXCLUDE
ls -altr
terraform init -upgrade

total 44
drwxrwxr-x 7 student student 4096 Jan  9 14:44 ..
-rw-rw-r-- 1 student student  247 Jan  9 14:44 provider.tf
-rw-rw-r-- 1 student student  665 Jan  9 14:44 resources.tf
-rw-r--r-- 1 student student 1406 Jan  9 14:44 .terraform.lock.hcl
-rw-rw-r-- 1 student student 5939 Jan  9 14:45 terraform.tfstate
-rw-rw-r-- 1 student student 1116 Jan  9 15:29 vars.tf
-rw-rw-r-- 1 student student   23 Jan  9 15:29 terraform.tfvars
-rw-rw-r-- 1 student student 1070 Jan  9 15:29 aws_instance.tf
drwxrwxr-x 2 student student 4096 Jan  9 15:29 .
-rw-rw-r-- 1 student student  335 Jan  9 15:29 outputs.tf

Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/aws versions matching "~> 4.0"...
- Finding latest version of hashicorp/tls...
- Using previously-installed hashicorp/aws v4.49.0
- Installing hashicorp/tls v4.0.4...
- Installed hashicorp/tls v4.0.4 (signed by HashiCorp)

Terraform has made some changes to the provider dependency selections recorded
in the .terrafor

In [12]:
#EXCLUDE
TF_APPLY

aws_vpc.main_vpc: Refreshing state... [id=vpc-0a27f3377fe36f60e]
aws_subnet.vpc_subnets[0]: Refreshing state... [id=subnet-063d030c0be1e1b6e]
aws_subnet.vpc_subnets[1]: Refreshing state... [id=subnet-0ff96eff4a1656bbd]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_instance.example[0] will be created
  + resource "aws_instance" "example" {
      + ami                                  = "ami-00a3e4424e9ab3e56"
      + arn                                  = (known after apply)
      + associate_public_ip_address          = (known after apply)
      + availability_zone                    = (known after apply)
      + cpu_core_count                       = (known after apply)
      + cpu_threads_per_core                 = (known after apply)
      + disable_api_stop                     = (known after apply)
      + disable_api_termi

          + tags                  = (known after apply)
          + throughput            = (known after apply)
          + volume_id             = (known after apply)
          + volume_size           = (known after apply)
          + volume_type           = (known after apply)
        }
    }

  # aws_instance.example[1] will be created
  + resource "aws_instance" "example" {
      + ami                                  = "ami-00a3e4424e9ab3e56"
      + arn                                  = (known after apply)
      + associate_public_ip_address          = (known after apply)
      + availability_zone                    = (known after apply)
      + cpu_core_count                       = (known after apply)
      + cpu_threads_per_core                 = (known after apply)
      + disable_api_stop                     = (known after apply)
      + disable_api_termination              = (known after apply)
      + ebs_optimized                        = (known after apply)
      + get_

          + tags                  = (known after apply)
          + throughput            = (known after apply)
          + volume_id             = (known after apply)
          + volume_size           = (known after apply)
          + volume_type           = (known after apply)
        }
    }

  # aws_key_pair.generated_key will be created
  + resource "aws_key_pair" "generated_key" {
      + arn             = (known after apply)
      + fingerprint     = (known after apply)
      + id              = (known after apply)
      + key_name        = "LAB4b-key"
      + key_name_prefix = (known after apply)
      + key_pair_id     = (known after apply)
      + key_type        = (known after apply)
      + public_key      = (known after apply)
      + tags_all        = {
          + "Environment" = "Terraform Introduction"
        }
    }

  # aws_security_group.secgroup-ssh will be created
  + resource "aws_security_group" "secgroup-ssh" {
      + arn                    = (known after app

: 1

#### There is an error in the above config - work in progress (regression)

We've achieved most of the lab, which is around the use of Terraform Maps

### 10. Investigate the *terraform state* sub-command

You can list the resources available in the current state using the ```terraform state list``` command.

Look at the state of one of the listed resources using command ```terraform state show <resource>```, e.g.
- ```terraform state show aws_vpc.main_vpc```
- ```terraform state show aws_instance.example[0]```

### 11. Cleanup

In [13]:
TF_DESTROY

tls_private_key.mykey: Refreshing state... [id=e92e6d88710e7dd7c9bde48de0d8b282862feeef]
aws_key_pair.generated_key: Refreshing state... [id=LAB4b-key]
aws_vpc.main_vpc: Refreshing state... [id=vpc-0a27f3377fe36f60e]
aws_subnet.vpc_subnets[0]: Refreshing state... [id=subnet-063d030c0be1e1b6e]
aws_subnet.vpc_subnets[1]: Refreshing state... [id=subnet-0ff96eff4a1656bbd]
aws_security_group.secgroup-ssh: Refreshing state... [id=sg-049d574d73c3307d1]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # aws_key_pair.generated_key will be destroyed
  - resource "aws_key_pair" "generated_key" {
      - arn         = "arn:aws:ec2:us-west-1:816376574968:key-pair/LAB4b-key" -> null
      - fingerprint = "6c:68:e1:3f:3d:5c:6f:85:29:56:68:74:e5:8a:76:0b" -> null
      - id          = "LAB4b-key" -> null
      - key_name    = "LAB4b-key" -> null
     

      - enable_dns64                                   = false -> null
      - enable_resource_name_dns_a_record_on_launch    = false -> null
      - enable_resource_name_dns_aaaa_record_on_launch = false -> null
      - id                                             = "subnet-0ff96eff4a1656bbd" -> null
      - ipv6_native                                    = false -> null
      - map_customer_owned_ip_on_launch                = false -> null
      - map_public_ip_on_launch                        = false -> null
      - owner_id                                       = "816376574968" -> null
      - private_dns_hostname_type_on_launch            = "ip-name" -> null
      - tags                                           = {
          - "LabName" = "4b.TerraformMaps"
          - "Name"    = "subnet-2"
        } -> null
      - tags_all                                       = {
          - "Environment" = "Terraform Introduction"
          - "LabName"     = "4b.TerraformMaps"
          - "

    EOT -> null
aws_security_group.secgroup-ssh: Destroying... [id=sg-049d574d73c3307d1]
aws_subnet.vpc_subnets[1]: Destroying... [id=subnet-0ff96eff4a1656bbd]
aws_subnet.vpc_subnets[0]: Destroying... [id=subnet-063d030c0be1e1b6e]
aws_key_pair.generated_key: Destroying... [id=LAB4b-key]
aws_key_pair.generated_key: Destruction complete after 1s
tls_private_key.mykey: Destroying... [id=e92e6d88710e7dd7c9bde48de0d8b282862feeef]
tls_private_key.mykey: Destruction complete after 0s
aws_subnet.vpc_subnets[1]: Destruction complete after 1s
aws_subnet.vpc_subnets[0]: Destruction complete after 1s
aws_security_group.secgroup-ssh: Destruction complete after 2s
aws_vpc.main_vpc: Destroying... [id=vpc-0a27f3377fe36f60e]
aws_vpc.main_vpc: Destruction complete after 1s

Destroy complete! Resources: 6 destroyed.



To destroy the formerly created AWS vpc, and all subnets.

<hr/>

# Summary

In this Exercise we looked at the use of the *map* type.

We used this type to map from
- region to ami image
- region to availability zones

We then created some VM instances and output their subnet, zone information

# Solutions

Solutions are available in the *github* repo at ```https://github.com/mjbright/tf-scenarios``` under Solutions at https://github.com/mjbright/tf-scenarios/tree/main/Solutions/lab4b


# **Stretch Goal 1:** Dynamic Blocks

- Add a security group to your VMs, using a dynamic block to specify each ingress rule to allow incoming traffic on ports 22 and 8080
- Investigate the state of the dynamic block you created
- Verify that you can ssh into your instances

You might want to refer to https://github.com/hashicorp/terraform-guides/tree/master/infrastructure-as-code/terraform-0.12-examples/dynamic-blocks-and-splat-expressions for some hints

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg)
<img src="../../../static/images/LOGO_v2_CROPPED.jpg" width="200" /> -->
<img src="../images/LOGO_v2_CROPPED.jpg" width="200" />

In [14]:
EXEC cd $NB_DIR
EXCL_FN_LAB_ENV

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

# Pragma --EXCLUDE--SECTION--

<div class="alert alert-danger"> <b>NOTE</b> Save notebook first </div>
<!-- ## **NOTE:** Save notebook first -->

<div id="END" /> <a href="#TOP" > TOP OF PAGE </a>  <!-- must appear after EXCLUDE pragma !! -->

In [15]:
set +x
. ~/scripts/nbtool.rc
pwd
#NB_DIR=$PWD
#echo NB=$NB
#echo NB_DIR=$NB_DIR
#set -x
set +x
EXCL_FN_FILTER_NOTEBOOK $NB

TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
/home/student/src/mjbright.tf-scenarios-private/ServeUpLabs/content/tf-labs/4b.TerraformMaps

==== Filtering current notebook [/home/student/src/mjbright.tf-scenarios-private/ServeUpLabs/content/tf-labs/4b.Terraform

In [ ]:
exit 1

In [ ]:
cd $NB_DIR
nbtool.py -f index.ipynb
ls -altr *.ipynb

jupyter-nbconvert --to markdown index.ipynb.filtered.ipynb

echo "$FRONTMATTER"          > index.md
cat index.ipynb.filtered.md >> index.md
mv  index.ipynb.filtered.md index.ipynb.filtered.md.bak
ls -altr *.md